### Basic Import

In [67]:
import sqlalchemy

In [68]:
sqlalchemy.__version__ 

'2.0.22'

In [69]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [70]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world','SMILE ALL'"))
    print(result.all())

2023-11-18 23:21:25,217 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:25,218 INFO sqlalchemy.engine.Engine select 'hello world','SMILE ALL'
2023-11-18 23:21:25,219 INFO sqlalchemy.engine.Engine [generated in 0.00269s] ()
[('hello world', 'SMILE ALL')]
2023-11-18 23:21:25,222 INFO sqlalchemy.engine.Engine ROLLBACK


In [71]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],
    )
    conn.commit()

2023-11-18 23:21:25,243 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:25,244 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-11-18 23:21:25,246 INFO sqlalchemy.engine.Engine [generated in 0.00358s] ()
2023-11-18 23:21:25,249 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-11-18 23:21:25,250 INFO sqlalchemy.engine.Engine [generated in 0.00121s] [(1, 1), (2, 4)]
2023-11-18 23:21:25,253 INFO sqlalchemy.engine.Engine COMMIT


In [72]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-11-18 23:21:25,272 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:25,274 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-11-18 23:21:25,275 INFO sqlalchemy.engine.Engine [cached since 0.02676s ago] [(6, 8), (9, 10)]
2023-11-18 23:21:25,278 INFO sqlalchemy.engine.Engine COMMIT


In [73]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    #print(type(result))
    for row in result:
        #print(type(row))
        print(f"x: {row.x}  y: {row.y}")
    print(result.all())
        

2023-11-18 23:21:25,302 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:25,303 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-11-18 23:21:25,305 INFO sqlalchemy.engine.Engine [generated in 0.00376s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
[]
2023-11-18 23:21:25,308 INFO sqlalchemy.engine.Engine ROLLBACK


In [74]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-11-18 23:21:25,328 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:25,330 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2023-11-18 23:21:25,332 INFO sqlalchemy.engine.Engine [generated in 0.00369s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-11-18 23:21:25,335 INFO sqlalchemy.engine.Engine ROLLBACK


In [75]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

2023-11-18 23:21:25,364 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:25,365 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-11-18 23:21:25,367 INFO sqlalchemy.engine.Engine [cached since 0.118s ago] [(11, 12), (13, 14)]
2023-11-18 23:21:25,371 INFO sqlalchemy.engine.Engine COMMIT


### Database MetaData

In [76]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

In [77]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

In [78]:

user_table.c

In [79]:
user_table.c.id


Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False)

In [80]:
user_table.c.keys()

['id', 'name', 'fullname']

In [81]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

Creating second table:

In [82]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

When using the ForeignKey object within a Column definition, we can omit the datatype for that Column; it is automatically inferred from that of the related column, in the above example the Integer datatype of the user_account.id column.

In [83]:
metadata_obj.create_all(engine)

2023-11-18 23:21:25,600 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:25,602 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-11-18 23:21:25,605 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-18 23:21:25,608 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2023-11-18 23:21:25,610 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-18 23:21:25,613 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-11-18 23:21:25,617 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-18 23:21:25,621 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2023-11-18 23:21:25,623 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-18 23:21:25,626 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2023-11-18 23:21:25,628 INFO sqlalchemy.engine.Engine [no key 0.00228s] ()
2023-11-18 23:21:25,633 INFO sqlalchemy.engine.Engine 
C

In [84]:
#metadata_obj.drop_all(bind=engine)

### Establishing a Declarative Base 

In [85]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

In [86]:
Base.metadata

MetaData()

In [87]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

#User and Address are ORM Mapped Classes
class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [88]:
Base.metadata.create_all(engine)

2023-11-18 23:21:25,765 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:25,768 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-11-18 23:21:25,770 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-18 23:21:25,774 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-11-18 23:21:25,776 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-18 23:21:25,782 INFO sqlalchemy.engine.Engine COMMIT


### Insert statement

In [89]:
from sqlalchemy import insert
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")

In [90]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [91]:
compiled = stmt.compile()

In [92]:
print(compiled)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


In [93]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2023-11-18 23:21:25,924 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:25,926 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-11-18 23:21:25,929 INFO sqlalchemy.engine.Engine [generated in 0.00499s] ('spongebob', 'Spongebob Squarepants')
2023-11-18 23:21:25,936 INFO sqlalchemy.engine.Engine COMMIT


In [94]:
result.inserted_primary_key

(1,)

In [95]:
print(insert(user_table))

INSERT INTO user_account (id, name, fullname) VALUES (:id, :name, :fullname)


common usage without .value clause:

In [96]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2023-11-18 23:21:26,076 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:26,080 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-11-18 23:21:26,085 INFO sqlalchemy.engine.Engine [generated in 0.00980s] [('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star')]
2023-11-18 23:21:26,094 INFO sqlalchemy.engine.Engine COMMIT


Advanced usage:

bindparam is used to match the input column "username" with table column "name"

In [97]:
from sqlalchemy import select, bindparam
scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {
                "username": "spongebob",
                "email_address": "spongebob@sqlalchemy.org",
            },
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2023-11-18 23:21:26,160 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:26,165 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = ?), ?)
2023-11-18 23:21:26,171 INFO sqlalchemy.engine.Engine [generated in 0.01073s] [('spongebob', 'spongebob@sqlalchemy.org'), ('sandy', 'sandy@sqlalchemy.org'), ('sandy', 'sandy@squirrelpower.org')]
2023-11-18 23:21:26,183 INFO sqlalchemy.engine.Engine COMMIT


#### INSERT…FROM SELECT


In [98]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address



##### user_account.name || :name_1: 

The || operator is a concatenation operator in SQL, which combines two strings into one. Here, it is combining the name field from the user_account table and the value of :name_1 parameter. 

The :name_1 is a placeholder for a parameter value that you pass in when you execute the query.

##### AS anon_1: 
This is giving an alias to the result of user_account.name || :name_1. It's just a temporary name for that column in this specific query. It's often used when the column name would be long or complicated. In this case, 'anon_1' doesn't seem to be used later in the query, it's likely just a placeholder or a convention used by the SQL Alchemy ORM (Object Relational Mapper) to handle the results of the concatenation.

### Select statement

In [99]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == "spongebob")
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


In [100]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2023-11-18 23:21:26,309 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:26,311 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2023-11-18 23:21:26,315 INFO sqlalchemy.engine.Engine [generated in 0.00621s] ('spongebob',)
(1, 'spongebob', 'Spongebob Squarepants')
2023-11-18 23:21:26,320 INFO sqlalchemy.engine.Engine ROLLBACK


##### ORM usage

In [101]:
from sqlalchemy.orm import Session

stmt = select(User).where(User.name == "spongebob")
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2023-11-18 23:21:26,353 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:26,370 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2023-11-18 23:21:26,372 INFO sqlalchemy.engine.Engine [generated in 0.00186s] ('spongebob',)
(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)
2023-11-18 23:21:26,376 INFO sqlalchemy.engine.Engine ROLLBACK


In [102]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


In [103]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


The above are not the same!

there is an important difference when we select from a full entity such as User, as opposed to user_table, which is that the entity itself is returned as a single element within each row. That is, when we fetch rows from the above statement, as there is only the User entity in the list of things to fetch, we get back Row objects that have only one element, which contain instances of the User class:

In [104]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [105]:
row = session.execute(select(User)).first()
row

2023-11-18 23:21:26,486 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:26,489 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2023-11-18 23:21:26,491 INFO sqlalchemy.engine.Engine [generated in 0.00230s] ()


(User(id=1, name='spongebob', fullname='Spongebob Squarepants'),)

A highly recommended convenience method of achieving the same result as above is to use the Session.scalars() method to execute the statement directly; this method will return a ScalarResult object that delivers the first “column” of each row at once, in this case, instances of the User class:

In [106]:
user = session.scalars(select(User)).first()
user

2023-11-18 23:21:26,532 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2023-11-18 23:21:26,535 INFO sqlalchemy.engine.Engine [cached since 0.04545s ago] ()


User(id=1, name='spongebob', fullname='Spongebob Squarepants')

In [107]:
row = session.execute(select(User.name, User.fullname)).first()
row

2023-11-18 23:21:26,567 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2023-11-18 23:21:26,568 INFO sqlalchemy.engine.Engine [generated in 0.00196s] ()


('spongebob', 'Spongebob Squarepants')

Mixed approach:

In [108]:
session.execute(
    select(User.name, Address).where(User.id == Address.user_id).order_by(Address.id)
).all()

2023-11-18 23:21:26,606 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2023-11-18 23:21:26,609 INFO sqlalchemy.engine.Engine [generated in 0.00225s] ()


[('spongebob', Address(id=1, email_address='spongebob@sqlalchemy.org')),
 ('sandy', Address(id=2, email_address='sandy@sqlalchemy.org')),
 ('sandy', Address(id=3, email_address='sandy@squirrelpower.org'))]

In [109]:
from sqlalchemy import func, cast
stmt = select(
    ("Username: " + user_table.c.name).label("username"),
).order_by(user_table.c.name)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.username}")

2023-11-18 23:21:26,651 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:26,654 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2023-11-18 23:21:26,656 INFO sqlalchemy.engine.Engine [generated in 0.00453s] ('Username: ',)
Username: patrick
Username: sandy
Username: spongebob
2023-11-18 23:21:26,660 INFO sqlalchemy.engine.Engine ROLLBACK


### Where clause

In [110]:
print(
    select(address_table.c.email_address)
    .where(user_table.c.name == "squidward")
    .where(address_table.c.user_id == user_table.c.id)
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


In [111]:
print(
    select(address_table.c.email_address).where(
        user_table.c.name == "squidward",
        address_table.c.user_id == user_table.c.id,
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


#### using AND OR

In [112]:
from sqlalchemy import and_, or_
print(
    select(Address.email_address).where(
        and_(
            or_(User.name == "squidward", User.name == "sandy"),
            Address.user_id == User.id,
        )
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


#### Using JOIN

In [113]:
print(
    select(user_table.c.name, address_table.c.email_address).join_from(
        user_table, address_table
    )
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


In [114]:
print(select(user_table.c.name, address_table.c.email_address).join(address_table))

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


#### Using explicit FROM Clause for more specific queries

In [115]:
print(select(address_table.c.email_address).select_from(user_table).join(address_table))

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


#### Using functions


In [116]:
from sqlalchemy import func
print(select(func.count("*")).select_from(user_table))

SELECT count(:count_2) AS count_1 
FROM user_account


#### Using ON Clause when there is no or more than one foreign keys

In [117]:
print(
    select(address_table.c.email_address)
    .select_from(user_table)
    .join(address_table, user_table.c.id == address_table.c.user_id)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


#### OUTER and FULL JOIN

In [118]:
print(select(user_table).join(address_table, isouter=True))
print(select(user_table).join(address_table, full=True))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account LEFT OUTER JOIN address ON user_account.id = address.user_id
SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account FULL OUTER JOIN address ON user_account.id = address.user_id


#### ORDER BY and DESC/ASC

In [119]:
print(select(User).order_by(User.fullname.desc()))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account ORDER BY user_account.fullname DESC


#### GROUPBY and HAVING

In [120]:
with engine.connect() as conn:
    result = conn.execute(
        select(User.name, func.count(Address.id).label("count"))
        .join(Address)
        .group_by(User.name)
        .having(func.count(Address.id) > 1)
    )
    print(result.all())

2023-11-18 23:21:26,972 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:26,976 INFO sqlalchemy.engine.Engine SELECT user_account.name, count(address.id) AS count 
FROM user_account JOIN address ON user_account.id = address.user_id GROUP BY user_account.name 
HAVING count(address.id) > ?
2023-11-18 23:21:26,979 INFO sqlalchemy.engine.Engine [generated in 0.00514s] (1,)
[('sandy', 2)]
2023-11-18 23:21:26,984 INFO sqlalchemy.engine.Engine ROLLBACK


### Data manipulation with ORM

In [121]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")

In [122]:
squidward

User(id=None, name='squidward', fullname='Squidward Tentacles')

In [123]:
session = Session(engine)

In [124]:
session.add(squidward)
session.add(krabs)

When we have pending objects, we can see this state by looking at a collection on the Session called Session.new:

In [125]:
session.new

IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles'), User(id=None, name='ehkrabs', fullname='Eugene H. Krabs')])

In [126]:
session.flush()

2023-11-18 23:21:27,170 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-18 23:21:27,174 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2023-11-18 23:21:27,176 INFO sqlalchemy.engine.Engine [generated in 0.00023s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('squidward', 'Squidward Tentacles')
2023-11-18 23:21:27,181 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2023-11-18 23:21:27,183 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('ehkrabs', 'Eugene H. Krabs')


In [128]:
print(squidward.id)
print(krabs.id)

4
5


#### Getting Objects by Primary Key from the Identity Map

In [129]:
some_squidward = session.get(User, 4)
some_squidward

User(id=4, name='squidward', fullname='Squidward Tentacles')

In [130]:
session.commit()

2023-11-18 23:24:18,650 INFO sqlalchemy.engine.Engine COMMIT


In [131]:
sandy = session.execute(select(User).filter_by(name="sandy")).scalar_one()

2023-11-19 07:27:50,341 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-19 07:27:50,348 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2023-11-19 07:27:50,353 INFO sqlalchemy.engine.Engine [cached since 2.919e+04s ago] ('sandy',)


In [132]:
sandy

User(id=2, name='sandy', fullname='Sandy Cheeks')

In [133]:
sandy in session.dirty

False

If we alter the attributes of this object, the Session tracks this change:

In [134]:
sandy.fullname = "Sandy Squirrel"
sandy in session.dirty

True

When the Session next emits a flush, an UPDATE will be emitted that updates this value in the database. As mentioned previously, a flush occurs automatically before we emit any SELECT, using a behavior known as autoflush. We can query directly for the User.fullname column from this row and we will get our updated value back:

In [135]:
sandy_fullname = session.execute(select(User.fullname).where(User.id == 2)).scalar_one()
print(sandy_fullname)

2023-11-19 07:37:21,376 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.id = ?
2023-11-19 07:37:21,380 INFO sqlalchemy.engine.Engine [generated in 0.00407s] ('Sandy Squirrel', 2)
2023-11-19 07:37:21,389 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2023-11-19 07:37:21,391 INFO sqlalchemy.engine.Engine [generated in 0.00250s] (2,)
Sandy Squirrel


In [136]:
sandy in session.dirty

False

#### Deleting ORM Objects using the Unit of Work pattern

In [137]:
patrick = session.get(User, 3)

2023-11-19 07:38:47,798 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2023-11-19 07:38:47,800 INFO sqlalchemy.engine.Engine [generated in 0.00232s] (3,)


In [138]:
session.delete(patrick)

In [139]:
session.execute(select(User).where(User.name == "patrick")).first()

2023-11-19 07:39:20,041 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2023-11-19 07:39:20,044 INFO sqlalchemy.engine.Engine [generated in 0.00366s] (3,)
2023-11-19 07:39:20,050 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2023-11-19 07:39:20,054 INFO sqlalchemy.engine.Engine [generated in 0.00346s] (3,)
2023-11-19 07:39:20,062 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2023-11-19 07:39:20,064 INFO sqlalchemy.engine.Engine [cached since 2.988e+04s ago] ('patrick',)


#### ORM Bulk INSERT Statements¶

In [140]:
from sqlalchemy import insert
session.execute(
    insert(User),
    [
        {"name": "spongebob", "fullname": "Spongebob Squarepants"},
        {"name": "sandy", "fullname": "Sandy Cheeks"},
        {"name": "patrick", "fullname": "Patrick Star"},
        {"name": "squidward", "fullname": "Squidward Tentacles"},
        {"name": "ehkrabs", "fullname": "Eugene H. Krabs"},
    ],
)

2023-11-19 07:53:58,848 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-11-19 07:53:58,851 INFO sqlalchemy.engine.Engine [generated in 0.00289s] [('spongebob', 'Spongebob Squarepants'), ('sandy', 'Sandy Cheeks'), ('patrick', 'Patrick Star'), ('squidward', 'Squidward Tentacles'), ('ehkrabs', 'Eugene H. Krabs')]


In [155]:
users = session.scalars(
    insert(User).returning(User),
    [
        {"name": "spongebob", "fullname": "Spongebob Squarepants"},
        {"name": "sandy", "fullname": "Sandy Cheeks"},
        {"name": "patrick", "fullname": "Patrick Star"},
        {"name": "squidward", "fullname": "Squidward Tentacles"},
        {"name": "ehkrabs", "fullname": "Eugene H. Krabs"},
    ],
)


2023-11-19 07:58:45,865 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?), (?, ?), (?, ?), (?, ?), (?, ?) RETURNING id, name, fullname
2023-11-19 07:58:45,869 INFO sqlalchemy.engine.Engine [cached since 152.2s ago (insertmanyvalues) 1/1 (unordered)] ('spongebob', 'Spongebob Squarepants', 'sandy', 'Sandy Cheeks', 'patrick', 'Patrick Star', 'squidward', 'Squidward Tentacles', 'ehkrabs', 'Eugene H. Krabs')


In [156]:
print(users.all())

[User(id=36, name='spongebob', fullname='Spongebob Squarepants'), User(id=37, name='sandy', fullname='Sandy Cheeks'), User(id=38, name='patrick', fullname='Patrick Star'), User(id=39, name='squidward', fullname='Squidward Tentacles'), User(id=40, name='ehkrabs', fullname='Eugene H. Krabs')]


#### Sending NULL values in ORM bulk INSERT statements

In [159]:
session.execute(
    insert(User),
    [
        {
            "name": "name_a",
            "fullname": "Employee A",
            "species": "Squid",
        },
        {
            "name": "name_b",
            "fullname": "Employee B",
            "species": "Squirrel",
        },
        {
            "name": "name_c",
            "fullname": None,
            "species": None,
        },
        {
            "name": "name_d",
            "fullname": "Employee D",
            "species": "Bluefish",
        },
    ],
)

2023-11-19 08:05:08,561 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-11-19 08:05:08,563 INFO sqlalchemy.engine.Engine [cached since 669.7s ago] [('name_a', 'Employee A'), ('name_b', 'Employee B')]
2023-11-19 08:05:08,568 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name) VALUES (?)
2023-11-19 08:05:08,571 INFO sqlalchemy.engine.Engine [generated in 0.00303s] ('name_c',)
2023-11-19 08:05:08,575 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-11-19 08:05:08,579 INFO sqlalchemy.engine.Engine [generated in 0.00411s] ('name_d', 'Employee D')


Above, the bulk INSERT of four rows is broken into three separate statements, the second statement reformatted to not refer to the NULL column for the single parameter dictionary that contains a None value. This default behavior may be undesirable when many rows in the dataset contain random NULL values, as it causes the “executemany” operation to be broken into a larger number of smaller operations; particularly when relying upon insertmanyvalues to reduce the overall number of statements, this can have a bigger performance impact.

To disable the handling of None values in the parameters into separate batches, pass the execution option render_nulls=True; this will cause all parameter dictionaries to be treated equivalently, assuming the same set of keys in each dictionary:

In [158]:
session.execute(
    insert(User).execution_options(render_nulls=True),
    [
        {
            "name": "name_a",
            "fullname": "Employee A",
            "species": "Squid",
        },
        {
            "name": "name_b",
            "fullname": "Employee B",
            "species": "Squirrel",
        },
        {
            "name": "name_c",
            "fullname": "Employee C",
            "species": None,
        },
        {
            "name": "name_d",
            "fullname": "Employee D",
            "species": "Bluefish",
        },
    ],
)

2023-11-19 08:04:51,019 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2023-11-19 08:04:51,019 INFO sqlalchemy.engine.Engine [cached since 652.2s ago] [('name_a', 'Employee A'), ('name_b', 'Employee B'), ('name_c', 'Employee C'), ('name_d', 'Employee D')]
